In [1]:
import pandas as pd

In [2]:
from address_match_proto import is_valid_street_simple

In [3]:
global ZIP_DF

In [4]:

file_path = 'ZIP_Locale_Detail.xls'

# Read the Excel file into a Pandas DataFrame
ZIP_DF = pd.read_excel(file_path, dtype=str)

# Display the DataFrame
ZIP_DF.head()

,AREA NAME,AREA CODE,DISTRICT NAME,DISTRICT NO,DELIVERY ZIPCODE,LOCALE NAME,PHYSICAL DELV ADDR,PHYSICAL CITY,PHYSICAL STATE,PHYSICAL ZIP,PHYSICAL ZIP 4
0,SOUTHERN,4G,PUERTO RICO,006,00601,ADJUNTAS,37 CALLE MUNOZ RIVERA,ADJUNTAS,PR,00601,9998
1,SOUTHERN,4G,PUERTO RICO,006,00602,AGUADA,5 AVE NATIVO ALERS,AGUADA,PR,00602,9998
2,SOUTHERN,4G,PUERTO RICO,006,00603,AGUADILLA,50 CARR 459 STE 1,AGUADILLA,PR,00603,9998
3,SOUTHERN,4G,PUERTO RICO,006,00604,RAMEY,100 AVE BORINQUEN,AGUADILLA,PR,00603,9996
4,SOUTHERN,4G,PUERTO RICO,006,00605,AGUADILLA,50 CARR 459 STE 1,AGUADILLA,PR,00603,9998


In [5]:
ZIP_DF.columns

Index(['AREA NAME', 'AREA CODE', 'DISTRICT NAME', 'DISTRICT NO',
       'DELIVERY ZIPCODE', 'LOCALE NAME', 'PHYSICAL DELV ADDR',
       'PHYSICAL CITY', 'PHYSICAL STATE', 'PHYSICAL ZIP', 'PHYSICAL ZIP 4'],
      dtype='object')

In [6]:
zip2_df = ZIP_DF[['PHYSICAL CITY', 'PHYSICAL STATE', 'PHYSICAL ZIP']]

In [7]:
zip2_df.head()

,PHYSICAL CITY,PHYSICAL STATE,PHYSICAL ZIP
0,ADJUNTAS,PR,00601
1,AGUADA,PR,00602
2,AGUADILLA,PR,00603
3,AGUADILLA,PR,00603
4,AGUADILLA,PR,00603


In [8]:
vdf=pd.read_csv('chaos_one.csv')

Index(['AREA NAME', 'AREA CODE', 'DISTRICT NAME', 'DISTRICT NO',
       'DELIVERY ZIPCODE', 'LOCALE NAME', 'PHYSICAL DELV ADDR',
       'PHYSICAL CITY', 'PHYSICAL STATE', 'PHYSICAL ZIP', 'PHYSICAL ZIP 4'],
      dtype='object')

In [9]:
class ZIP:

    def is_valid(self, zip_code):
        global ZIP_DF
        filtered_rows = ZIP_DF[ZIP_DF['DELIVERY ZIPCODE'] == zip_code]
        if filtered_rows.empty:
            return False
        else:
            return True
    
    def is_equal(self, zip_code, city):
        global ZIP_DF
        filtered_rows = ZIP_DF[ZIP_DF['DELIVERY ZIPCODE'] == zip_code]
        #print(filtered_rows['LOCALE NAME'].values[0])
        locale_name = filtered_rows['LOCALE NAME'].values[0]
        physical_city = filtered_rows['PHYSICAL CITY'].values[0]
        # print(locale_name, physical_city)
        if city.upper() == locale_name:
            return True

        if city.upper() == physical_city:
            return True
        else:
            return False
   
    def is_5_digits(self, zip_code):
        if len(zip_code) == 5:
            return True
        else:
            return False




In [10]:
def qb_validate_zip(df_row):
    #print(df_row.Zip, df_row.City)
    # Example usage:
    zip_instance = ZIP()
     # Test is_equal method
    qb_zip = str(df_row.Zip)
    qb_city = df_row.City

    zip_valid = zip_instance.is_valid(qb_zip)
    if not zip_valid:
        print(f"zip_valid result: {zip_valid}")
        return False
    # Test is_5_digits method
    digits_result = zip_instance.is_5_digits(qb_zip)
    #print(f"is_5_digits result: {digits_result}")

    if not digits_result:
        print(f"is_5_digits result: {digits_result}")
        return False

    
    zip_result = zip_instance.is_equal(qb_zip, qb_city)
    #print(f"is_equal result: {zip_result}")

    if not zip_result:
        print(f"City and ZIP is_equal result: {zip_result}")
        return False

    return True

In [11]:
def qb_validate_street(df_row):
    qb_zip = str(df_row.Zip)
    qb_street_address = df_row['Street Address']
    qb_state = df_row.State
    qb_city = df_row.City

    print(qb_street_address, qb_city, qb_state, qb_zip)
    a_status = is_valid_street_simple(qb_street_address, qb_city, qb_state, qb_zip)
    print(a_status)
    return a_status
    

In [12]:
bad_list_of_dicts = []
for i, my_df_row in vdf.iterrows():
    #print(type(my_df_row))
    #print(my_df_row)
    if qb_validate_zip(my_df_row):
        pass
    else:
        print('BIG PROBLEM')
        print(my_df_row)
        print('-'*30)
        bad_list_of_dicts.append(my_df_row.to_dict())
        continue
    
    if qb_validate_street(my_df_row):
        pass
    else:
        print('SMALL PROBLEM')
        print(my_df_row)
        print('-'*30)
        bad_list_of_dicts.append(my_df_row.to_dict())

4100 S Ash Grove Ave Sioux Falls SD 57103
True
4101 S Ash Grove Ave Sioux Falls SD 57103
True
4200 S Ash Grove Ave Sioux Falls SD 57103
True
4201 S Ash Grove Ave Sioux Falls SD 57103
True
4204 S Ash Grove Ave Sioux Falls SD 57103
True
4205 S Ash Grove Ave Sioux Falls SD 57103
True
4208 S Ash Grove Ave Sioux Falls SD 57103
True
4209 S Ash Grove Ave Sioux Falls SD 57103
True
4213 S Ash Grove Ave Sioux Falls SD 57103
True
4301 S Mountain Ash Dr Sioux Falls SD 57103
True
4304 S Ash Grove Ave Sioux Falls SD 57103
True
4304 S Mountain Ash Dr Sioux Falls SD 57103
True
4305 S Ash Grove Ave Sioux Falls SD 57103
True
4305 S Mountain Ash Dr Sioux Falls SD 57103
True
City and ZIP is_equal result: False
BIG PROBLEM
Vendor                  Lee Advertising
Company                 Lee Advertising
Street Address    4308 S Moutain Ash Dr
City                         Siox Falls
State                                SD
Country                             NaN
Zip                               57103
1099 Tra

In [13]:
bad_df = pd.DataFrame(bad_list_of_dicts)

In [14]:
bad_df

,Vendor,Company,Street Address,City,State,Country,Zip,1099 Tracking,Phone,Email,Attachments,Open Balance
0,Lee Advertising,Lee Advertising,4308 S Moutain Ash Dr,Siox Falls,SD,NaN,57103,NaN,(650) 554-4622,NaN,NaN,0.0
1,National Eye Care,National Eye Care,4312 S Ash Grove Ave,Sioux Falls,SD,NaN,57198,NaN,(800) 555-5555,"Nateyecare@intuit.com, pauliejones15@intuit.com",NaN,0.0
2,Norton Lumber and Building Materials,Norton Lumber and Building Materials,4813 S Mountain Ash Dr,Sioux Falls,SD,NaN,57103,NaN,(650) 363-6578,Materials@intuit.com,NaN,205.0
3,Squeaky Kleen Car Wash,NaN,4317 S Mounin Ash Dr,Sioux Falls,SD,NaN,57103,NaN,NaN,NaN,NaN,0.0
4,Tania's Nursery,Tania's Nursery,4320 S Ash Grove Ave,Sioux Falls,SD,USA,5713,NaN,(886) 554-2265,plantqueen@taniasnursery.com,NaN,0.0
5,Tim Philip Masonry,Tim Philip Masonry,4321 S Ash Grove Ave,Sioux Flls,SD,NaN,57103,NaN,(800) 556-1254,tim.philip@timphilipmasonry.com,NaN,0.0


In [15]:
my_df_row.to_dict()

{'Vendor': 'United States Treasury',
 'Company': nan,
 'Street Address': '4400 S Mountain Ash Dr',
 'City': 'Sioux Falls',
 'State': 'SD',
 'Country': nan,
 'Zip': 57103,
 '1099 Tracking': nan,
 'Phone': '(888) 555-6454',
 'Email': 'taxesaregreat@intuit.com',
 'Attachments': nan,
 'Open Balance': 0.0}

In [16]:
bad_df.to_csv('outliers.csv')